# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-81c563df6f-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Film Genre and directors explorative search") 

Consider the following exploratory information need:

> investigate the results concerning the different film genre over years and the directors for the cinema.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q25089`| Woody Allen       | node |
| `wd:Q3772`| Quentin Tarantino      | node |






Also consider

```
wd:Q25089 ?p ?obj .
```

is the BGP to retrieve all **properties of Woody Allen**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for films

2. Identify the BGP for director

3. Identify the BGP for film genre

4. Find how many western films are been released in each european country from 2010-01-01 to 2015-31-12 (the result set must be country IRI, label and #films).

5. Consider only films directed by Woody Allen or Quentin Tarantino. Then consider the date of publication of these films (take the first one if there are more than one date of publication). Return the year with more films release for the two director (the result set must be two elements and for each element there must be present the director IRI, the label of the IRI, the year and the number of films). 

6. Identify the BGP for Academy Awards 

7. Consider films directed by Woody Allen or Quentin Tarantino separately. Return the film which won more Academy Awards for the two directors (the result set must be two elements - one related to Allen and the other related to Tarantino -  and for each element there must be present the director IRI with its label, the film IRI with its label and the number of Academy awards won).

8. Consider the set of films directed by Allen or Tarantino. Return for each Academy Award the total number of nomination of these films (the result set must be a list of triples Academy Award IRI, label and #nomination).

## Task 1
Identify the BGP for films

In [24]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q25089 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('name', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P1150'), ('name', 'Regensburg Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1220'), ('name', 'Internet Broadway Database person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P

In [25]:
# query example
#checking the objects of properties to find a clue 
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objname
WHERE {
   # bind something
   wd:Q25089 ?p ?obj .
   
   # get the label
   ?p sc:name ?name.
    ?obj sc:name ?objname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after'), ('obj', 'http://www.wikidata.org/entity/Q349357'), ('objname', 'Woody Herman')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('name', 'spouse'), ('obj', 'http://www.wikidata.org/entity/Q1140914'), ('objname', 'Soon-Yi Previn')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('name', 'child'), ('obj', 'http://www.wikidata.org/entity/Q22673707'), ('objname', 'Bechet Dumaine Allen')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('name', 'child'), ('obj', 'http://www.wikidata.org/entity/Q22673708'), ('objname', 'Manzie Tio Allen')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth'), ('obj', 'http://www.wikidata.org/entity/Q18426'), ('objname', 'The Bronx')]
[('p', 'http://www.wikidata.org/prop/direct/P451'), ('name', 'unmarried partner'), ('obj', 'http://www.wikidata.org/entity/Q102642'), ('objname', 'Diane Keaton')]
[('p', 'http://www.wikidata.org/prop/direct/P16

In [26]:
#[('p', 'http://www.wikidata.org/prop/direct/P800'), ('name', 'notable work')
#wd:Q25089	Woody Allen	node
#finding some films to find the BGP
queryString = """
SELECT DISTINCT  ?obj ?objname
WHERE {
   # bind something
   wd:Q25089 wdt:P800 ?obj .
   
   # get the label
 
    ?obj sc:name ?objname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q206124'), ('objname', 'Midnight in Paris')]
[('obj', 'http://www.wikidata.org/entity/Q971865'), ('objname', "What's Up, Tiger Lily?")]
[('obj', 'http://www.wikidata.org/entity/Q1004531'), ('objname', 'Bullets Over Broadway')]
[('obj', 'http://www.wikidata.org/entity/Q1354109'), ('objname', 'Mighty Aphrodite')]
[('obj', 'http://www.wikidata.org/entity/Q729026'), ('objname', 'Broadway Danny Rose')]
[('obj', 'http://www.wikidata.org/entity/Q733677'), ('objname', 'Match Point')]
[('obj', 'http://www.wikidata.org/entity/Q740143'), ('objname', 'Husbands and Wives')]
[('obj', 'http://www.wikidata.org/entity/Q806092'), ('objname', 'Bananas')]
[('obj', 'http://www.wikidata.org/entity/Q1321004'), ('objname', 'You Will Meet a Tall Dark Stranger')]
[('obj', 'http://www.wikidata.org/entity/Q14511869'), ('objname', 'Magic in the Moonlight')]
[('obj', 'http://www.wikidata.org/entity/Q614009'), ('objname', 'Anything Else')]
[('obj', 'http://www.wikidat

In [149]:
#[('p', 'http://www.wikidata.org/prop/direct/P800'), ('name', 'notable work')
#('obj', 'http://www.wikidata.org/entity/Q206124'), ('objname', 'Midnight in Paris')]

#wd:Q25089	Woody Allen	node
#checking film properties 
queryString = """
SELECT DISTINCT  ?p ?pname ?res ?resname
WHERE {
   # bind something
   wd:Q25089 wdt:P800 ?obj .
   ?obj ?p ?res .
   # get the label
 
    ?p sc:name ?pname.
    ?res sc:name ?resname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('res', 'http://www.wikidata.org/entity/Q22963890'), ('resname', 'Angel Caban')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('res', 'http://www.wikidata.org/entity/Q19668406'), ('resname', 'Yolonda Ross')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('res', 'http://www.wikidata.org/entity/Q184103'), ('resname', 'Christina Ricci')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('res', 'http://www.wikidata.org/entity/Q448005'), ('resname', 'David Ortiz')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('res', 'http://www.wikidata.org/entity/Q126455'), ('resname', 'Peter Hobbs')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('res', 'http://www.wikidata.org/entity/Q132616'), ('resname', 'Naomi Watts')]
[('p', 'http://www.wikidata.org/prop/dire

Final query for this task

In [154]:
#[('p', 'http://www.wikidata.org/prop/direct/P800'), ('name', 'notable work')
#('obj', 'http://www.wikidata.org/entity/Q206124'), ('objname', 'Midnight in Paris')]
#checking film properties 
#wd:Q25089	Woody Allen	node
queryString = """
SELECT DISTINCT  ?class ?classname
WHERE {
   # bind something
   wd:Q25089 wdt:P800 ?film .
   ?film wdt:P31 ?class .
   # get the label
 
    ?class sc:name ?classname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q11424'), ('classname', 'film')]
[('class', 'http://www.wikidata.org/entity/Q5398426'), ('classname', 'television series')]
2


## Task 2
Identify the BGP for director

In [30]:
#wd:Q25089	Woody Allen	node
#wdt:P106	occupation	predicate
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q25089 wdt:P106 ?p .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q10800557'), ('name', 'film actor')]
[('p', 'http://www.wikidata.org/entity/Q15981151'), ('name', 'jazz musician')]
[('p', 'http://www.wikidata.org/entity/Q18844224'), ('name', 'science fiction writer')]
[('p', 'http://www.wikidata.org/entity/Q1930187'), ('name', 'journalist')]
[('p', 'http://www.wikidata.org/entity/Q214917'), ('name', 'playwright')]
[('p', 'http://www.wikidata.org/entity/Q245068'), ('name', 'comedian')]
[('p', 'http://www.wikidata.org/entity/Q2526255'), ('name', 'film director')]
[('p', 'http://www.wikidata.org/entity/Q28389'), ('name', 'screenwriter')]
[('p', 'http://www.wikidata.org/entity/Q3282637'), ('name', 'film producer')]
[('p', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer')]
[('p', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
[('p', 'http://www.wikidata.org/entity/Q482980'), ('name', 'author')]
[('p', 'http://www.wikidata.org/entity/Q639669'), ('name', 'musician')]
[('p', 'http://

In [155]:
#wd:Q25089	Woody Allen	node
#wdt:P106	occupation	predicate
#wdt:P279	subclass	predicate
queryString = """
SELECT DISTINCT ?result  ?resultname
WHERE {
   # bind something
   wd:Q25089 wdt:P106 ?occupation.
   ?occupation wdt:P279 ?result .
   # get the label
   ?result sc:name ?resultname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('result', 'http://www.wikidata.org/entity/Q28225718'), ('resultname', 'science fiction author')]
[('result', 'http://www.wikidata.org/entity/Q47541952'), ('resultname', 'producer')]
[('result', 'http://www.wikidata.org/entity/Q33999'), ('resultname', 'actor')]
[('result', 'http://www.wikidata.org/entity/Q3455803'), ('resultname', 'director')]
[('result', 'http://www.wikidata.org/entity/Q36180'), ('resultname', 'writer')]
[('result', 'http://www.wikidata.org/entity/Q482980'), ('resultname', 'author')]
[('result', 'http://www.wikidata.org/entity/Q483501'), ('resultname', 'artist')]
[('result', 'http://www.wikidata.org/entity/Q639669'), ('resultname', 'musician')]
[('result', 'http://www.wikidata.org/entity/Q3391743'), ('resultname', 'visual artist')]
[('result', 'http://www.wikidata.org/entity/Q101546515'), ('resultname', 'woodwind instrument player')]
[('result', 'http://www.wikidata.org/entity/Q2500638'), ('resultname', 'creator')]
[('result', 'http://www.wikidata.org/entity/

Final query for this task

In [157]:
#wd:Q25089	Woody Allen	node
#wdt:P106	occupation	predicate
#wdt:P279	subclass	predicate
queryString = """
SELECT DISTINCT ?result  ?resultname
WHERE {
   # bind something
   wd:Q25089 wdt:P106 ?occupation.
   ?occupation wdt:P279 ?result .
   # get the label
   ?result sc:name ?resultname.
   FILTER(REGEX(?resultname ,'director'))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('result', 'http://www.wikidata.org/entity/Q3455803'), ('resultname', 'director')]
1


## Task 3
Identify the BGP for film genre

In [3]:
#[('p', 'http://www.wikidata.org/prop/direct/P800'), ('name', 'notable work')
#('obj', 'http://www.wikidata.org/entity/Q206124'), ('objname', 'Midnight in Paris')] 
#wd:Q25089	Woody Allen	node

#using the node that we founf to find the genre
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q206124 ?p ?obj .
   # get the label

    ?p sc:name ?name.
    FILTER(REGEX(?name , '[Gg]enre'))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
1


In [163]:
#[('p', 'http://www.wikidata.org/prop/direct/P800'), ('name', 'notable work')
#('obj', 'http://www.wikidata.org/entity/Q206124'), ('objname', 'Midnight in Paris')] 
#wd:Q25089	Woody Allen	node
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
#wdt:P279	subclass	predicate
#using the node that we find to find the genre
queryString = """
SELECT DISTINCT ?class ?classname
WHERE {
   # bind something
   wd:Q206124 wdt:P136 ?genre .
   ?genre wdt:P279 ?class .
   # get the label

    ?class sc:name ?classname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q1054574'), ('classname', 'romance film')]
[('class', 'http://www.wikidata.org/entity/Q11424'), ('classname', 'film')]
[('class', 'http://www.wikidata.org/entity/Q157443'), ('classname', 'comedy film')]
[('class', 'http://www.wikidata.org/entity/Q24925'), ('classname', 'science fiction')]
[('class', 'http://www.wikidata.org/entity/Q2973181'), ('classname', 'speculative fiction film')]
[('class', 'http://www.wikidata.org/entity/Q132311'), ('classname', 'fantasy')]
6


Final query for this task

In [166]:
#wdt:P31	instance of	predicate
#[('p', 'http://www.wikidata.org/prop/direct/P800'), ('name', 'notable work')
#('obj', 'http://www.wikidata.org/entity/Q206124'), ('objname', 'Midnight in Paris')] 
#wd:Q25089	Woody Allen	node
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
#wdt:P279	subclass	predicate
#using the node that we find to find the genre
queryString = """
SELECT DISTINCT ?result ?resultname
WHERE {
   # bind something
   wd:Q206124 wdt:P136 ?genre .
   ?genre wdt:P279 ?class .
   ?class wdt:P31 ?result .
   # get the label

    ?result sc:name ?resultname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('result', 'http://www.wikidata.org/entity/Q108329096'), ('resultname', 'genre by art form')]
[('result', 'http://www.wikidata.org/entity/Q201658'), ('resultname', 'film genre')]
[('result', 'http://www.wikidata.org/entity/Q20076756'), ('resultname', 'speculative fiction genre')]
3


## Task 4
Find how many western films are been released in each european country from 2010-01-01 to 2015-31-12 (the result set must be country IRI, label and #films).

In [169]:
#[('result', 'http://www.wikidata.org/entity/Q201658'), ('resultname', 'film genre')]
#wdt:P31	instance of	predicate
#finding all film genres that contain western
queryString = """
SELECT DISTINCT  ?genre ?genrename
WHERE {
   # bind something
   ?genre wdt:P31 wd:Q201658 .
   
   # get the label
    ?genre sc:name ?genrename.
    
    FILTER (REGEX(?genrename , 'Western'))
}  order by ?sname
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('genre', 'http://www.wikidata.org/entity/Q268854'), ('genrename', 'Red Western')]
[('genre', 'http://www.wikidata.org/entity/Q4235011'), ('genrename', 'Space Western')]
[('genre', 'http://www.wikidata.org/entity/Q172980'), ('genrename', 'Western film')]
[('genre', 'http://www.wikidata.org/entity/Q491158'), ('genrename', 'Revisionist Western')]
[('genre', 'http://www.wikidata.org/entity/Q6804274'), ('genrename', 'meat pie Western')]
[('genre', 'http://www.wikidata.org/entity/Q5461696'), ('genrename', 'Florida Western')]
[('genre', 'http://www.wikidata.org/entity/Q4674071'), ('genrename', 'Acid Western')]
[('genre', 'http://www.wikidata.org/entity/Q8066387'), ('genrename', 'Zapata Western')]
8


In [7]:
#[('class', 'http://www.wikidata.org/entity/Q11424'), ('classname', 'film')]
#finding Country  properties for films
queryString = """
SELECT DISTINCT  ?p ?pname ?obj ?objname
WHERE {
   # bind something
   wd:Q11424 ?p ?obj .
   # get the label
   ?p sc:name ?pname.
   ?obj sc:name ?objname.
   FILTER(REGEX(?objname ,'country'))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P495'), ('objname', 'country of origin')]
1


In [22]:
#[('class', 'http://www.wikidata.org/entity/Q11424'), ('classname', 'film')]
#finding date  properties for films
queryString = """
SELECT DISTINCT  ?p ?pname ?obj ?objname
WHERE {
   # bind something
   wd:Q11424 ?p ?obj .
   # get the label
   ?p sc:name ?pname.
   ?obj sc:name ?objname.
   FILTER(REGEX(?objname ,'date'))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P577'), ('objname', 'publication date')]
1


In [23]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'),
#('obj', 'http://www.wikidata.org/entity/P495'), ('objname', 'country of origin')]
#[('class', 'http://www.wikidata.org/entity/Q11424'), ('classname', 'film')]
#[('result', 'http://www.wikidata.org/entity/Q201658'), ('resultname', 'film genre')]
#wdt:P31	instance of	predicate
#finding all film genres that contain western
#[('genre', 'http://www.wikidata.org/entity/Q172980'), ('genrename', 'Western film')]
#('obj', 'http://www.wikidata.org/entity/P577'), ('objname', 'publication date')]

#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]

#FOR EACH FILM FOUNDS THE COUNTRY OF ORIGIN AND PUBLICATION DATE AND FILTERS THE DATES 
queryString = """
SELECT DISTINCT    ?film ?filmname ?country ?countryname
WHERE {
   # bind something
   #the film having the western genre 
   ?film wdt:P136 wd:Q172980 ;
         wdt:P495 ?country;
         wdt:P577 ?date.
         
         
   
   # get the label
   ?fil sc:name ?filmname.
   ?country sc:name ?countryname
   #filtring the date
    FILTER (YEAR(?date)>2009 && YEAR(?date)<2016)
} 
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q18225084'), ('filmname', 'software version identifier'), ('country', 'http://www.wikidata.org/entity/Q30'), ('countryname', 'United States of America')]
[('film', 'http://www.wikidata.org/entity/Q18225084'), ('filmname', 'tabular software version'), ('country', 'http://www.wikidata.org/entity/Q30'), ('countryname', 'United States of America')]
[('film', 'http://www.wikidata.org/entity/Q18225084'), ('filmname', 'has edition or translation'), ('country', 'http://www.wikidata.org/entity/Q30'), ('countryname', 'United States of America')]
[('film', 'http://www.wikidata.org/entity/Q18225084'), ('filmname', 'Fonsi Nieto'), ('country', 'http://www.wikidata.org/entity/Q30'), ('countryname', 'United States of America')]
[('film', 'http://www.wikidata.org/entity/Q18225084'), ('filmname', "Al Stewart's albums in chronological order"), ('country', 'http://www.wikidata.org/entity/Q30'), ('countryname', 'United States of America')]
[('film', 'http:/

In [11]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'),
#('obj', 'http://www.wikidata.org/entity/P495'), ('objname', 'country of origin')]
#[('class', 'http://www.wikidata.org/entity/Q11424'), ('classname', 'film')]
#[('result', 'http://www.wikidata.org/entity/Q201658'), ('resultname', 'film genre')]
#wdt:P31	instance of	predicate
#finding all film genres that contain western
#[('genre', 'http://www.wikidata.org/entity/Q172980'), ('genrename', 'Western film')]
#('obj', 'http://www.wikidata.org/entity/P577'), ('objname', 'publication date')]



#CHECKING FOR THE CONTINENT OF THE COuNTRY
queryString = """
SELECT DISTINCT  ?p ?pname ?n ?name
WHERE {
   # bind something
   ?film wdt:P136 wd:Q172980 ;
         wdt:P495 ?country;
         wdt:P577 ?date.
    ?country ?p ?obj .
    ?obj wdt:P31 ?n .
         
   
   # get the label
   
  
    ?p sc:name ?pname.
    ?n sc:name ?name
   FILTER(REGEX(?pname ,'continent'))
    FILTER (YEAR(?date)>2009 && YEAR(?date)<2016)
} 
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent'), ('n', 'http://www.wikidata.org/entity/Q5107'), ('name', 'continent')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent'), ('n', 'http://www.wikidata.org/entity/Q2418896'), ('name', 'part of the world')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent'), ('n', 'http://www.wikidata.org/entity/Q855697'), ('name', 'subcontinent')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent'), ('n', 'http://www.wikidata.org/entity/Q82794'), ('name', 'geographic region')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent'), ('n', 'http://www.wikidata.org/entity/Q55833'), ('name', 'supercontinent')]
5


In [188]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'),
#('obj', 'http://www.wikidata.org/entity/P495'), ('objname', 'country of origin')]
#[('class', 'http://www.wikidata.org/entity/Q11424'), ('classname', 'film')]
#[('result', 'http://www.wikidata.org/entity/Q201658'), ('resultname', 'film genre')]
#wdt:P31	instance of	predicate
#finding all film genres that contain western
#[('genre', 'http://www.wikidata.org/entity/Q172980'), ('genrename', 'Western film')]
#('obj', 'http://www.wikidata.org/entity/P577'), ('objname', 'publication date')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'),
#('pname', 'continent'), ('n', 'http://www.wikidata.org/entity/Q5107'), ('name', 'continent')]



#FINDING CONTINENT NAMES AND CODES
queryString = """
SELECT DISTINCT   ?continent  ?continentname
WHERE {
   # bind something
   ?film wdt:P136 wd:Q172980 ;
         wdt:P495 ?country;
         wdt:P577 ?date.
    ?country wdt:P30 ?continent .
    
         
   
   # get the label
   
  
    ?continent  sc:name ?continentname.
    
   
    FILTER (YEAR(?date)>2009 && YEAR(?date)<2016)
} 
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q15'), ('continentname', 'Africa')]
[('continent', 'http://www.wikidata.org/entity/Q46'), ('continentname', 'Europe')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('continentname', 'Asia')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('continentname', 'North America')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('continentname', 'Insular Oceania')]
[('continent', 'http://www.wikidata.org/entity/Q5401'), ('continentname', 'Eurasia')]
6


In [190]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'),
#('obj', 'http://www.wikidata.org/entity/P495'), ('objname', 'country of origin')]
#[('class', 'http://www.wikidata.org/entity/Q11424'), ('classname', 'film')]
#[('result', 'http://www.wikidata.org/entity/Q201658'), ('resultname', 'film genre')]
#wdt:P31	instance of	predicate
#finding all film genres that contain western
#[('genre', 'http://www.wikidata.org/entity/Q172980'), ('genrename', 'Western film')]
#('obj', 'http://www.wikidata.org/entity/P577'), ('objname', 'publication date')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'),
#('pname', 'continent'), ('n', 'http://www.wikidata.org/entity/Q5107'), ('name', 'continent')]
#[('continent', 'http://www.wikidata.org/entity/Q46'), ('continentname', 'Europe')]

#AADING BEING EUROPEAN CONDITION 

queryString = """
SELECT DISTINCT   ?country  ?countryname
WHERE {
   # bind something
   ?film wdt:P136 wd:Q172980 ;
         wdt:P495 ?country;
         wdt:P577 ?date.
    ?country wdt:P30 wd:Q46. 
    
         
   
   # get the label
  
    ?country  sc:name ?countryname.

   
    FILTER (YEAR(?date)>2009 && YEAR(?date)<2016)
} 
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryname', 'France')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryname', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('countryname', 'Russia')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryname', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q221'), ('countryname', 'North Macedonia')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryname', 'Spain')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryname', 'Belgium')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('countryname', 'Sweden')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryname', 'Denmark')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryname', 'Austria')]
[('country', 'http://www.wikidata.org/entity/Q43'), ('countryname', 'Turkey')]
11


Final query for this task

In [13]:
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
#[('genre', 'http://www.wikidata.org/entity/Q172980'), ('genrename', 'Western film')]
#('obj', 'http://www.wikidata.org/entity/P495'), ('objname', 'country of origin')]
#('obj', 'http://www.wikidata.org/entity/P577'), ('objname', 'publication date')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'),
#[('continent', 'http://www.wikidata.org/entity/Q46'), ('continentname', 'Europe')]


#FINAL QUERY

queryString = """
SELECT DISTINCT   ?country  ?countryname (count (?film) AS ?numberOfFilms)
WHERE {
   # bind something
   ?film wdt:P136 wd:Q172980;
       wdt:P495 ?country;
       wdt:P577 ?date.
    ?country wdt:P30 wd:Q46. 
    
   # get the label
  
    ?country  sc:name ?countryname.

   
    FILTER (YEAR(?date)>2009 && YEAR(?date)<2016)
} group by ?country  ?countryname
"""

print("Results")
x=run_query(queryString)
 

Results
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryname', 'Germany'), ('numberOfFilms', '7')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryname', 'Belgium'), ('numberOfFilms', '2')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('countryname', 'Sweden'), ('numberOfFilms', '2')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryname', 'Spain'), ('numberOfFilms', '1')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryname', 'France'), ('numberOfFilms', '3')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryname', 'Austria'), ('numberOfFilms', '3')]
[('country', 'http://www.wikidata.org/entity/Q221'), ('countryname', 'North Macedonia'), ('numberOfFilms', '1')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryname', 'Denmark'), ('numberOfFilms', '2')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryname', 'United Kingdom'), ('numberOfFilms', '4')]
[('country', 'http://www.wikidata.org/ent

## Task 5
Consider only films directed by Woody Allen or Quentin Tarantino. Then consider the date of publication of these films (take the first one if there are more than one date of publication). Return the year with more films release for the two director (the result set must be two elements and for each element there must be present the director IRI, the label of the IRI, the year and the number of films).

In [14]:
#CHECKING FILM PROPERTIES TO FIND THE DIRECTOR PROPERTY 
queryString = """
SELECT DISTINCT   ?c ?cname
WHERE {
    VALUES ?director {wd:Q25089 wd:Q3772} .
    wd:Q11424 ?p ?c.
    ?p sc:name ?pname.
    ?c sc:name ?cname
    filter(regex(?cname , 'director'))
    
}

LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/P344'), ('cname', 'director of photography')]
[('c', 'http://www.wikidata.org/entity/P57'), ('cname', 'director')]
2


In [51]:
#('c', 'http://www.wikidata.org/entity/P57'), ('cname', 'director')]
#wd:Q25089	Woody Allen	node
#wd:Q3772	Quentin Tarantino	node
#('obj', 'http://www.wikidata.org/entity/P577'), ('objname', 'publication date')] 

#FINDING ALL THE MOVIES DIRECTED BY Woody Allen or Quentin Tarantino AND THEIR PUBLICATION YEAR
queryString = """
SELECT DISTINCT  ?film ?name  YEAR(?date)
WHERE {
   VALUES ?director {wd:Q25089 wd:Q3772} .
   ?film wdt:P57 wd:Q25089;
        wdt:P577 ?date .
   
    ?film sc:name ?name  
}

LIMIT 100
"""
print("Results")
x=run_query(queryString)



Results
[('film', 'http://www.wikidata.org/entity/Q3986392'), ('name', 'The Concert for New York City'), ('callret-2', '2001')]
[('film', 'http://www.wikidata.org/entity/Q206124'), ('name', 'Midnight in Paris'), ('callret-2', '2011')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('name', 'Alice'), ('callret-2', '1990')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('name', 'Alice'), ('callret-2', '1991')]
[('film', 'http://www.wikidata.org/entity/Q971865'), ('name', "What's Up, Tiger Lily?"), ('callret-2', '1966')]
[('film', 'http://www.wikidata.org/entity/Q1004531'), ('name', 'Bullets Over Broadway'), ('callret-2', '1994')]
[('film', 'http://www.wikidata.org/entity/Q1004531'), ('name', 'Bullets Over Broadway'), ('callret-2', '1995')]
[('film', 'http://www.wikidata.org/entity/Q1354109'), ('name', 'Mighty Aphrodite'), ('callret-2', '1995')]
[('film', 'http://www.wikidata.org/entity/Q1354109'), ('name', 'Mighty Aphrodite'), ('callret-2', '1996')]
[('film', 'http://www.wik

In [387]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), 
#('c', 'http://www.wikidata.org/entity/P57'), ('cname', 'director')]
#wd:Q25089	Woody Allen	node
#wd:Q3772	Quentin Tarantino	node
#('obj', 'http://www.wikidata.org/entity/P577'), ('objname', 'publication date')]

#FINDING ALL THE MOVIES DIRECTED BY Woody Allen or Quentin Tarantino AND THEIR FIRST PUBLICATION YEAR
queryString = """

SELECT DISTINCT (min(YEAR(?date) ) as ?year) ?name 
WHERE {
 VALUES ?director { wd:Q3772} .
    ?director sc:name ?directorname.
   ?film wdt:P57 wd:Q3772;
        wdt:P577 ?date .
        
   
    ?film sc:name ?name  
}group by  ?name  
order by ?year
limit 100
"""
print("Results")
x=run_query(queryString)



Results
[('year', '1987'), ('name', "My Best Friend's Birthday")]
[('year', '1992'), ('name', 'Reservoir Dogs')]
[('year', '1994'), ('name', 'Pulp Fiction')]
[('year', '1995'), ('name', 'Four Rooms')]
[('year', '1997'), ('name', 'Jackie Brown')]
[('year', '2003'), ('name', 'Kill Bill: The Whole Bloody Affair')]
[('year', '2003'), ('name', 'Kill Bill Volume 1')]
[('year', '2004'), ('name', 'Kill Bill Volume 2')]
[('year', '2005'), ('name', 'Grave Danger')]
[('year', '2005'), ('name', 'Grave Danger: Part 2')]
[('year', '2005'), ('name', 'Grave Danger: Part 1')]
[('year', '2005'), ('name', 'Sin City')]
[('year', '2007'), ('name', 'Grindhouse')]
[('year', '2007'), ('name', 'Death Proof')]
[('year', '2009'), ('name', 'Inglourious Basterds')]
[('year', '2012'), ('name', 'Django Unchained')]
[('year', '2015'), ('name', 'The Hateful Eight')]
[('year', '2019'), ('name', 'Once Upon a Time in Hollywood')]
18


In [52]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), 
#('c', 'http://www.wikidata.org/entity/P57'), ('cname', 'director')]
#wd:Q25089	Woody Allen	node
#wd:Q3772	Quentin Tarantino	node
#('obj', 'http://www.wikidata.org/entity/P577'), ('objname', 'publication date')]

#FINDING ALL THE MOVIES DIRECTED BY Woody Allen  AND THEIR FIRST PUBLICATION YEAR
queryString = """

select ?director ?directorname ?year ?number 
{
            VALUES ?director {wd:Q25089 } .
            ?director sc:name ?directorname.
    {
    select (count(?name) as?number ) ?year 
    {
    
    {
        SELECT DISTINCT (min(YEAR(?date) ) as ?year) ?name 
        WHERE {
            
           ?film wdt:P57 wd:Q25089;
            wdt:P577 ?date .
             ?film sc:name ?name  
            }group by  ?name  
            order by ?year
            limit 100
            
        } 
}group by ?year 
}
}

"""
print("Results")
x=run_query(queryString)



Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('year', '1966'), ('number', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('year', '1980'), ('number', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('year', '1984'), ('number', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('year', '1998'), ('number', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('year', '2011'), ('number', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('year', '1982'), ('number', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('year', '1986'), ('number', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('year', '1988'), ('number', '1')]


In [389]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), 
#('c', 'http://www.wikidata.org/entity/P57'), ('cname', 'director')]
#wd:Q25089	Woody Allen	node
#wd:Q3772	Quentin Tarantino	node
#('obj', 'http://www.wikidata.org/entity/P577'), ('objname', 'publication date')]

#FINDING ALL THE MOVIES DIRECTED BY Woody Allen  AND THEIR FIRST PUBLICATION YEAR
queryString = """

select ?director ?directorname ?year ?number 
{
            VALUES ?director {wd:Q3772 } .
            ?director sc:name ?directorname.
    {
    select (count(?name) as?number ) ?year 
    {
    
    {
        SELECT DISTINCT (min(YEAR(?date) ) as ?year) ?name 
        WHERE {
            
           ?film wdt:P57 wd:Q3772;
            wdt:P577 ?date .
             ?film sc:name ?name  
            }group by  ?name  
            order by ?year
            limit 100
            
        } 
}group by ?year 
order by desc(?number )
}
}order by desc(?number)

"""
print("Results")
x=run_query(queryString)



Results
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorname', 'Quentin Tarantino'), ('year', '2005'), ('number', '4')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorname', 'Quentin Tarantino'), ('year', '2003'), ('number', '2')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorname', 'Quentin Tarantino'), ('year', '2007'), ('number', '2')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorname', 'Quentin Tarantino'), ('year', '1995'), ('number', '1')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorname', 'Quentin Tarantino'), ('year', '1987'), ('number', '1')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorname', 'Quentin Tarantino'), ('year', '2004'), ('number', '1')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorname', 'Quentin Tarantino'), ('year', '2012'), ('number', '1')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorname', 'Quentin Tarantin

In [401]:
queryString = """

SELECT * WHERE
{ 
{
   select ?director ?directorname ?year ?number 
{
            VALUES ?director {wd:Q3772 } .
            ?director sc:name ?directorname.
    {
    select (count(?name) as?number ) ?year 
    {
    
    {
        SELECT DISTINCT (min(YEAR(?date) ) as ?year) ?name 
        WHERE {
            
           ?film wdt:P57 wd:Q3772;
            wdt:P577 ?date .
             ?film sc:name ?name  
            }group by  ?name  
            order by ?year
            limit 100
            
        } 
}group by ?year 
}
}order by desc(?number)
limit 1 
}
UNION
{
    select ?director ?directorname ?year ?number 
{
            VALUES ?director {wd:Q25089 } .
            ?director sc:name ?directorname.
    {
    select (count(?name) as?number ) ?year 
    {
    
    {
        SELECT DISTINCT (min(YEAR(?date) ) as ?year) ?name 
        WHERE {
            
           ?film wdt:P57 wd:Q25089;
            wdt:P577 ?date .
             ?film sc:name ?name  
            }group by  ?name  
            order by ?year
            limit 100
            
        } 
}group by ?year 
}
}order by desc(?number)
limit 1 
        
}
}group by ?year 
limit 2


"""
print("Results")
x=run_query(queryString)


Results
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorname', 'Quentin Tarantino'), ('year', '2005'), ('number', '4')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('year', '1971'), ('number', '2')]
2


Final query for this task

In [53]:
#AS THE TASK DESCRIPTION WAS AMBIGUOUS AND I COULD NOT FIND OUT WHICH OF BELOW QUERIES IS ASSUMED I DID IN THIS WAY
# FIRST ASSUME WAS RETURN THE YEAR THAT SUM OF THEIR WORKS WAS THE MAX WHICH IS 2005 THAT TRENTINO HAD 4 FILMS 
#AND Allen DIRECTED ONLY ONE . BUT IT DID NOT SEEM RIGHT 
#SO I RETURNED THE YEARS THAT EACH ONE HAD MORE WORKS 
#FOR TERENTINO IT WAS NOT A PROBLEM BECAUSE IN 2005 HE HAD 4 FILMS 
#BUT FOR ALLEN HE HAD IN MAX 2 FILMS IN A YEAR 
#AND THERE WAS 5 CASES THAT ALLEN HAD 2 FILMS IN A YEAR (1971-1994-1987-2001 ,..)
#SO AS THE TASK ASKED ONLY TWO ELEMENTS (ONE FOR EACH DIRECTOE ) I DECIDED TO RETURN THE FIRST RECORD FOR 
#ALLEN .
queryString = """

SELECT * WHERE
{ 
{
   select ?director ?directorname ?year ?number 
{
            VALUES ?director {wd:Q3772 } .
            ?director sc:name ?directorname.
    {
    select (count(?name) as?number ) ?year 
    {
    
    {
        SELECT DISTINCT (min(YEAR(?date) ) as ?year) ?name 
        WHERE {
            
           ?film wdt:P57 wd:Q3772;
            wdt:P577 ?date .
             ?film sc:name ?name  
            }group by  ?name  
            order by ?year
            limit 100
            
        } 
}group by ?year 
}
}order by desc(?number)
limit 1 
}
UNION
{
    select ?director ?directorname ?year ?number 
{
            VALUES ?director {wd:Q25089 } .
            ?director sc:name ?directorname.
    {
    select (count(?name) as?number ) ?year 
    {
    
    {
        SELECT DISTINCT (min(YEAR(?date) ) as ?year) ?name 
        WHERE {
            
           ?film wdt:P57 wd:Q25089;
            wdt:P577 ?date .
             ?film sc:name ?name  
            }group by  ?name  
            order by ?year
            limit 100
            
        } 
}group by ?year 
}
}order by desc(?number)
limit 1 
        
}
}group by ?year 
limit 2


"""
print("Results")
x=run_query(queryString)


Results
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorname', 'Quentin Tarantino'), ('year', '2005'), ('number', '4')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('year', '1971'), ('number', '2')]
2


## Task 6
Identify the BGP for Academy Awards

In [18]:
#wd:Q25089	Woody Allen	node
queryString = """
SELECT DISTINCT  ?p ?pname
WHERE {
    wd:Q25089 ?p ?obj .
    
    ?p sc:name ?pname
   
}

LIMIT 100
"""
print("Results")
x=run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('pname', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('pname', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('pname', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('pname', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pname', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('pname', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P1150'), ('pname', 'Regensburg Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('pname', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1220'), ('pname', 'Internet Broadway Database person ID')]
[('p', 'http://www.wikidata.org/pr

In [133]:
#[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')] 
#wd:Q25089	Woody Allen	node
queryString = """
SELECT DISTINCT  ?award ?awardname
WHERE {
    wd:Q25089 wdt:P1411 ?award.
    
    ?award sc:name ?awardname
   
}

LIMIT 100
"""
print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/entity/Q103360'), ('pname', 'Academy Award for Best Director')]
[('p', 'http://www.wikidata.org/entity/Q41417'), ('pname', 'Academy Award for Best Writing, Original Screenplay')]
[('p', 'http://www.wikidata.org/entity/Q103916'), ('pname', 'Academy Award for Best Actor')]
[('p', 'http://www.wikidata.org/entity/Q1377772'), ('pname', 'European Film Award for Best Non-European Film')]
4


In [135]:
#[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')] 
#[('p', 'http://www.wikidata.org/entity/Q103360'), ('pname', 'Academy Award for Best Director')]
#wd:Q25089	Woody Allen	node
#wdt:P31	instance of	predicate
queryString = """
SELECT DISTINCT  ?var ?varname
WHERE {
    wd:Q25089 wdt:P1411 ?award.
    ?award wdt:P31 ?var.
    
    ?var sc:name ?varname
   
}

LIMIT 100
"""
print("Results")
x=run_query(queryString)


Results
[('var', 'http://www.wikidata.org/entity/Q96474679'), ('varname', 'award for best direction')]
[('var', 'http://www.wikidata.org/entity/Q19020'), ('varname', 'Academy Awards')]
[('var', 'http://www.wikidata.org/entity/Q223740'), ('varname', 'European Film Awards')]
[('var', 'http://www.wikidata.org/entity/Q96474691'), ('varname', 'award for best leading actor')]
[('var', 'http://www.wikidata.org/entity/Q96474700'), ('varname', 'award for best screenplay')]
5


Final query for this task

In [73]:
#[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')] 
#[('p', 'http://www.wikidata.org/entity/Q103360'), ('pname', 'Academy Award for Best Director')]
#wd:Q25089	Woody Allen	node
#wdt:P31	instance of	predicate
queryString = """
SELECT DISTINCT  ?var ?varname
WHERE {
    wd:Q25089 wdt:P1411 ?award.
    ?award wdt:P31 ?var.
    
    ?var sc:name ?varname
    FILTER(REGEX(?varname , '[Aa]cademy'))
   
}

LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('var', 'http://www.wikidata.org/entity/Q19020'), ('varname', 'Academy Awards')]
1


## Task 7
Consider films directed by Woody Allen or Quentin Tarantino separately. Return the film which won more Academy Awards for the two directors (the result set must be two elements - one related to Allen and the other related to Tarantino -  and for each element there must be present the director IRI with its label, the film IRI with its label and the number of Academy awards won).

In [210]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), 
#('c', 'http://www.wikidata.org/entity/P57'), ('cname', 'director')]
#wd:Q25089	Woody Allen	node
#wd:Q3772	Quentin Tarantino	node


#FINDING ALL THE MOVIES DIRECTED BY Woody Allen or Quentin Tarantino
queryString = """
SELECT DISTINCT  ?film ?name
WHERE {
   VALUES ?director {wd:Q25089 wd:Q3772} .
   ?film wdt:P57 wd:Q25089 .
   
    ?film sc:name ?name  
}

LIMIT 100
"""
print("Results")
x=run_query(queryString)



Results
[('film', 'http://www.wikidata.org/entity/Q3986392'), ('name', 'The Concert for New York City')]
[('film', 'http://www.wikidata.org/entity/Q206124'), ('name', 'Midnight in Paris')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('name', 'Alice')]
[('film', 'http://www.wikidata.org/entity/Q971865'), ('name', "What's Up, Tiger Lily?")]
[('film', 'http://www.wikidata.org/entity/Q1004531'), ('name', 'Bullets Over Broadway')]
[('film', 'http://www.wikidata.org/entity/Q1354109'), ('name', 'Mighty Aphrodite')]
[('film', 'http://www.wikidata.org/entity/Q729026'), ('name', 'Broadway Danny Rose')]
[('film', 'http://www.wikidata.org/entity/Q733677'), ('name', 'Match Point')]
[('film', 'http://www.wikidata.org/entity/Q740143'), ('name', 'Husbands and Wives')]
[('film', 'http://www.wikidata.org/entity/Q806092'), ('name', 'Bananas')]
[('film', 'http://www.wikidata.org/entity/Q1321004'), ('name', 'You Will Meet a Tall Dark Stranger')]
[('film', 'http://www.wikidata.org/entity/Q14511869'

In [216]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), 
#('c', 'http://www.wikidata.org/entity/P57'), ('cname', 'director')]
#wd:Q25089	Woody Allen	node
#wd:Q3772	Quentin Tarantino	node
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')] 
#[('var', 'http://www.wikidata.org/entity/Q19020'), ('varname', 'Academy Awards')]

#FINDING ALL THE MOVIES DIRECTED BY Woody Allen or Quentin Tarantino AND THEIR ACADEMY AWARDS
queryString = """
SELECT DISTINCT  ?name ?awardname 
WHERE {
   VALUES ?director {wd:Q25089 wd:Q3772} .
   ?film wdt:P57 wd:Q25089 ;
        wdt:P166 ?awards.
    ?awards wdt:P31 wd:Q19020 .
   
    ?film sc:name ?name  .
    ?awards sc:name ?awardname
}

LIMIT 100
"""
print("Results")
x=run_query(queryString)



Results
[('name', 'Bullets Over Broadway'), ('awardname', 'Academy Award for Best Supporting Actress')]
[('name', 'Mighty Aphrodite'), ('awardname', 'Academy Award for Best Supporting Actress')]
[('name', 'Blue Jasmine'), ('awardname', 'Academy Award for Best Actress')]
[('name', 'Vicky Cristina Barcelona'), ('awardname', 'Academy Award for Best Supporting Actress')]
[('name', 'Annie Hall'), ('awardname', 'Academy Award for Best Director')]
[('name', 'Annie Hall'), ('awardname', 'Academy Award for Best Picture')]
[('name', 'Annie Hall'), ('awardname', 'Academy Award for Best Actress')]
[('name', 'Hannah and Her Sisters'), ('awardname', 'Academy Award for Best Supporting Actress')]
[('name', 'Midnight in Paris'), ('awardname', 'Academy Award for Best Writing, Original Screenplay')]
[('name', 'Annie Hall'), ('awardname', 'Academy Award for Best Writing, Original Screenplay')]
[('name', 'Hannah and Her Sisters'), ('awardname', 'Academy Award for Best Supporting Actor')]
[('name', 'Hannah 

In [235]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), 
#('c', 'http://www.wikidata.org/entity/P57'), ('cname', 'director')]
#wd:Q25089	Woody Allen	node
#wd:Q3772	Quentin Tarantino	node
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')] 
#[('var', 'http://www.wikidata.org/entity/Q19020'), ('varname', 'Academy Awards')]

#FINDING ALL THE MOVIES DIRECTED BY Woody Allen or Quentin Tarantino AND count THEIR ACADEMY AWARDS c
queryString = """
SELECT DISTINCT  ?name ?awardname
WHERE {
   VALUES ?director {wd:Q25089 wd:Q3772} .
   ?film wdt:P57 wd:Q25089 ;
        wdt:P166 ?awards.
    ?awards wdt:P31 wd:Q19020 .
   
    ?film sc:name ?name  .
    ?awards sc:name ?awardname
} group by ?name

LIMIT 100
"""
print("Results")
x=run_query(queryString)



Results
[('name', 'Blue Jasmine'), ('awardname', 'Academy Award for Best Actress')]
[('name', 'Annie Hall'), ('awardname', 'Academy Award for Best Writing, Original Screenplay')]
[('name', 'Annie Hall'), ('awardname', 'Academy Award for Best Actress')]
[('name', 'Hannah and Her Sisters'), ('awardname', 'Academy Award for Best Supporting Actress')]
[('name', 'Bullets Over Broadway'), ('awardname', 'Academy Award for Best Supporting Actress')]
[('name', 'Mighty Aphrodite'), ('awardname', 'Academy Award for Best Supporting Actress')]
[('name', 'Annie Hall'), ('awardname', 'Academy Award for Best Picture')]
[('name', 'Vicky Cristina Barcelona'), ('awardname', 'Academy Award for Best Supporting Actress')]
[('name', 'Midnight in Paris'), ('awardname', 'Academy Award for Best Writing, Original Screenplay')]
[('name', 'Hannah and Her Sisters'), ('awardname', 'Academy Award for Best Supporting Actor')]
[('name', 'Annie Hall'), ('awardname', 'Academy Award for Best Director')]
[('name', 'Hannah 

In [254]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), 
#('c', 'http://www.wikidata.org/entity/P57'), ('cname', 'director')]
#wd:Q25089	Woody Allen	node
#wd:Q3772	Quentin Tarantino	node
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')] 
#[('var', 'http://www.wikidata.org/entity/Q19020'), ('varname', 'Academy Awards')]

#FINDING ALL THE MOVIES DIRECTED BY Woody Allen or Quentin Tarantino AND count THEIR ACADEMY AWARDS c
queryString = """
SELECT  ?director ?directorname ?film ?filmname (count (?awardname ) as ?number )
WHERE {
   VALUES ?director {wd:Q25089 wd:Q3772} .
   ?film wdt:P57 ?director ;
        wdt:P166 ?awards.
    ?awards wdt:P31 wd:Q19020 .
   
    ?film sc:name ?filmname  .
    ?awards sc:name ?awardname.
     ?director sc:name  ?directorname
     
} group by  ?director ?directorname ?film ?filmname 
order by desc(?number)


LIMIT 2
"""
print("Results")
x=run_query(queryString)



Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q233464'), ('filmname', 'Annie Hall'), ('number', '4')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q845057'), ('filmname', 'Hannah and Her Sisters'), ('number', '3')]
2


In [267]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), 
#('c', 'http://www.wikidata.org/entity/P57'), ('cname', 'director')]
#wd:Q25089	Woody Allen	node
#wd:Q3772	Quentin Tarantino	node
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')] 
#[('var', 'http://www.wikidata.org/entity/Q19020'), ('varname', 'Academy Awards')]

#FINDING ALL THE MOVIES DIRECTED BY Woody Allen or Quentin Tarantino AND count THEIR ACADEMY AWARDS c
queryString = """
SELECT  ?director ?directorname ?film ?filmname (count(?awardname) as ?number ) 
WHERE {
   VALUES ?director {wd:Q25089 wd:Q3772} .
   ?film wdt:P57 ?director ;
        wdt:P166 ?awards.
    ?awards wdt:P31 wd:Q19020 .
   
    ?film sc:name ?filmname  .
    ?awards sc:name ?awardname.
     ?director sc:name  ?directorname
     
} group by  ?director ?directorname ?film ?filmname 



LIMIT 20
"""
print("Results")
x=run_query(queryString)



Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q2907178'), ('filmname', 'Blue Jasmine'), ('number', '1')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorname', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q571032'), ('filmname', 'Django Unchained'), ('number', '2')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q1354109'), ('filmname', 'Mighty Aphrodite'), ('number', '1')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorname', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q104123'), ('filmname', 'Pulp Fiction'), ('number', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q845057'), ('filmname', 'Hannah and Her Sisters'), ('number', '3')]
[('director', 'http://

In [284]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), 
#('c', 'http://www.wikidata.org/entity/P57'), ('cname', 'director')]
#wd:Q25089	Woody Allen	node
#wd:Q3772	Quentin Tarantino	node
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')] 
#[('var', 'http://www.wikidata.org/entity/Q19020'), ('varname', 'Academy Awards')]

#FIRST APPROACH TO FIND THE ANSWER using union instead of value
queryString = """
SELECT * WHERE
{ 
{
   SELECT   ?director ?directorname ?film ?filmname (count(?awardname) as ?number ) 
    WHERE {
        VALUES ?director {  wd:Q25089 }
       ?film wdt:P57 ?director;
            wdt:P166 ?awards.
        ?awards wdt:P31 wd:Q19020 .
   
        ?film sc:name ?filmname  .
        ?awards sc:name ?awardname.
         ?director sc:name  ?directorname
     
        } group by  ?director ?directorname ?film ?filmname 
        order by desc(?number) 
        LIMIT 1
}
UNION
{
    SELECT ?director ?directorname ?film ?filmname (count(?awardname) as ?number ) 
    WHERE {
     VALUES ?director { wd:Q3772  }
       
       ?film wdt:P57 ?director ;
            wdt:P166 ?awards.
        ?awards wdt:P31 wd:Q19020 .
   
        ?film sc:name ?filmname  .
        ?awards sc:name ?awardname.
         ?director sc:name  ?directorname
     
        } group by  ?director ?directorname ?film ?filmname 
        order by desc(?number) 
        LIMIT 1
        
}
}
"""
print("Results")
x=run_query(queryString)



Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q233464'), ('filmname', 'Annie Hall'), ('number', '4')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorname', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q571032'), ('filmname', 'Django Unchained'), ('number', '2')]
2


Final query for this task

In [303]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), 
#('c', 'http://www.wikidata.org/entity/P57'), ('cname', 'director')]
#wd:Q25089	Woody Allen	node
#wd:Q3772	Quentin Tarantino	node
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')] 
#[('var', 'http://www.wikidata.org/entity/Q19020'), ('varname', 'Academy Awards')]

#FIRST APPROACH TO FIND THE ANSWER using union instead of value
queryString = """
SELECT * WHERE
{ 
{
   SELECT   ?director ?directorname ?film ?filmname (count(?awardname) as ?number ) 
    WHERE {
        VALUES ?director {  wd:Q25089 }
       ?film wdt:P57 ?director;
            wdt:P166 ?awards.
        ?awards wdt:P31 wd:Q19020 .
   
        ?film sc:name ?filmname  .
        ?awards sc:name ?awardname.
         ?director sc:name  ?directorname
     
        } group by  ?director ?directorname ?film ?filmname 
        order by desc(?number) 
        LIMIT 1
}
UNION
{
    SELECT ?director ?directorname ?film ?filmname (count(?awardname) as ?number ) 
    WHERE {
     VALUES ?director { wd:Q3772  }
       
       ?film wdt:P57 ?director ;
            wdt:P166 ?awards.
        ?awards wdt:P31 wd:Q19020 .
   
        ?film sc:name ?filmname  .
        ?awards sc:name ?awardname.
         ?director sc:name  ?directorname
     
        } group by  ?director ?directorname ?film ?filmname 
        order by desc(?number) 
        LIMIT 1
        
}
}
"""
print("Results")
x=run_query(queryString)



Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q233464'), ('filmname', 'Annie Hall'), ('number', '4')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorname', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q571032'), ('filmname', 'Django Unchained'), ('number', '2')]
2


## Task 8
Consider the set of films directed by Allen or Tarantino. Return for each Academy Award the total number of nomination of these films (the result set must be a list of triples Academy Award IRI, label and #nomination).

In [301]:
#[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')] 
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), 
#('c', 'http://www.wikidata.org/entity/P57'), ('cname', 'director')]
#wd:Q25089	Woody Allen	node
#wd:Q3772	Quentin Tarantino	node
#[('var', 'http://www.wikidata.org/entity/Q19020'), ('varname', 'Academy Awards')]

#FINDING ALL THE MOVIES DIRECTED BY Woody Allen or Quentin Tarantino AND count THEIR ACADEMY AWARDS c
queryString = """
SELECT DISTINCT  ?awards ?name  (count (?name) as ?numberofNOM)
WHERE {
   VALUES ?director {wd:Q25089 wd:Q3772} .
   ?film wdt:P57 ?director ;
        wdt:P1411 ?awards.
    ?awards wdt:P31 wd:Q19020 .

    ?awards sc:name ?name.
} group by?awards ?name 
order by desc(?numberofNOM)

LIMIT 100
"""
print("Results")
x=run_query(queryString)


Results
[('awards', 'http://www.wikidata.org/entity/Q41417'), ('name', 'Academy Award for Best Writing, Original Screenplay'), ('numberofNOM', '19')]
[('awards', 'http://www.wikidata.org/entity/Q106301'), ('name', 'Academy Award for Best Supporting Actress'), ('numberofNOM', '11')]
[('awards', 'http://www.wikidata.org/entity/Q103360'), ('name', 'Academy Award for Best Director'), ('numberofNOM', '9')]
[('awards', 'http://www.wikidata.org/entity/Q106291'), ('name', 'Academy Award for Best Supporting Actor'), ('numberofNOM', '7')]
[('awards', 'http://www.wikidata.org/entity/Q102427'), ('name', 'Academy Award for Best Picture'), ('numberofNOM', '6')]
[('awards', 'http://www.wikidata.org/entity/Q277751'), ('name', 'Academy Award for Best Production Design'), ('numberofNOM', '5')]
[('awards', 'http://www.wikidata.org/entity/Q131520'), ('name', 'Academy Award for Best Cinematography'), ('numberofNOM', '4')]
[('awards', 'http://www.wikidata.org/entity/Q103916'), ('name', 'Academy Award for Be

Final query for this task

In [302]:
#[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')] 
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), 
#('c', 'http://www.wikidata.org/entity/P57'), ('cname', 'director')]
#wd:Q25089	Woody Allen	node
#wd:Q3772	Quentin Tarantino	node
#[('var', 'http://www.wikidata.org/entity/Q19020'), ('varname', 'Academy Awards')]

#FINDING ALL THE MOVIES DIRECTED BY Woody Allen or Quentin Tarantino AND count THEIR ACADEMY AWARDS c
queryString = """
SELECT DISTINCT  ?awards ?name  (count (?name) as ?numberofNOM)
WHERE {
   VALUES ?director {wd:Q25089 wd:Q3772} .
   ?film wdt:P57 ?director ;
        wdt:P1411 ?awards.
    ?awards wdt:P31 wd:Q19020 .

    ?awards sc:name ?name.
} group by?awards ?name 
order by desc(?numberofNOM)

LIMIT 100
"""
print("Results")
x=run_query(queryString)


Results
[('awards', 'http://www.wikidata.org/entity/Q41417'), ('name', 'Academy Award for Best Writing, Original Screenplay'), ('numberofNOM', '19')]
[('awards', 'http://www.wikidata.org/entity/Q106301'), ('name', 'Academy Award for Best Supporting Actress'), ('numberofNOM', '11')]
[('awards', 'http://www.wikidata.org/entity/Q103360'), ('name', 'Academy Award for Best Director'), ('numberofNOM', '9')]
[('awards', 'http://www.wikidata.org/entity/Q106291'), ('name', 'Academy Award for Best Supporting Actor'), ('numberofNOM', '7')]
[('awards', 'http://www.wikidata.org/entity/Q102427'), ('name', 'Academy Award for Best Picture'), ('numberofNOM', '6')]
[('awards', 'http://www.wikidata.org/entity/Q277751'), ('name', 'Academy Award for Best Production Design'), ('numberofNOM', '5')]
[('awards', 'http://www.wikidata.org/entity/Q131520'), ('name', 'Academy Award for Best Cinematography'), ('numberofNOM', '4')]
[('awards', 'http://www.wikidata.org/entity/Q103916'), ('name', 'Academy Award for Be